In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1129089/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_9_1,0.606186,-25.911582,-73.336221,0.070275,0.046470,15.975878,52.074067,0.027428,0.076913,0.215570,0.206219,0.217247,136.137886,322.527625,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_9_0,0.600593,-24.631787,-71.034146,0.037332,0.047130,15.216136,50.461415,0.028399,0.077417,0.217095,0.194945,0.218785,136.109679,322.499418,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_8_9,0.575006,-22.024006,-62.989343,0.447664,0.050150,13.668044,44.825863,0.058303,0.079915,0.223941,0.143371,0.225684,135.985490,322.375229,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_8_8,0.570267,-21.962758,-62.735393,0.446659,0.050709,13.631685,44.647966,0.058409,0.080330,0.225186,0.133819,0.226939,135.963312,322.353051,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_8_7,0.566895,-21.979615,-62.429040,0.456027,0.051107,13.641692,44.433360,0.057420,0.080539,0.226068,0.127023,0.227827,135.947682,322.337421,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
5,model_1_9_5,0.563752,-27.201583,-73.816143,-2.230617,0.051478,16.741678,52.410262,0.095306,0.073546,0.226887,0.120687,0.228652,135.933219,322.322958,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
6,model_1_8_6,0.563741,-21.211958,-60.199168,0.462037,0.051479,13.185978,42.871288,0.056786,0.080730,0.226890,0.120665,0.228655,135.933168,322.322907,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
7,model_1_8_5,0.559218,-14.964754,-41.771923,0.504241,0.052013,9.477368,29.962620,0.052331,0.080534,0.228063,0.111549,0.229837,135.912541,322.302280,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
8,model_1_9_2,0.558645,-25.466318,-73.026465,-1.264915,0.052080,15.711550,51.857076,0.066817,0.077532,0.228211,0.110393,0.229987,135.909941,322.299680,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
9,model_1_9_6,0.558476,-25.626997,-71.109832,-2.376465,0.052100,15.806936,50.514435,0.099608,0.073601,0.228254,0.110054,0.230031,135.909178,322.298917,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
